In [ ]:
!pip install scikit-learn pandas nltk


In [3]:
import pandas as pd

# Cargar dataset
df = pd.read_csv("tmdb_5000_movies.csv")

# Mantener solo las columnas necesarias
df = df[['title', 'overview']]

# Mostrar las primeras filas
print(df.head())

# Reemplazar valores nulos en 'overview' con una cadena vacía
df['overview'] = df['overview'].fillna('')

from sklearn.feature_extraction.text import TfidfVectorizer

# Crear la matriz TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])

print("Matriz TF-IDF creada con dimensión:", tfidf_matrix.shape)

from sklearn.metrics.pairwise import cosine_similarity

# Calcular similitud de coseno entre las películas
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Crear una serie para mapear índices a títulos y viceversa
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

# Función de recomendación
def recommend_movies(title, cosine_sim=cosine_sim):
    if title not in indices:
        return f"La película '{title}' no está en la base de datos."

    # Obtener índice de la película seleccionada
    idx = indices[title]

    # Obtenemos los puntajes de similitud de coseno de la película seleccionada con todas las demás
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenamos las películas por puntaje de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenemos los índices de las 10 películas más similares
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]

    # Devolvemos los títulos de las películas similares
    return df['title'].iloc[movie_indices]
# Probar el recomendador
print("Películas recomendadas para 'The Godfather':\n")
print(recommend_movies("The Godfather"))


                                      title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  
0  In the 22nd century, a paraplegic Marine is di...  
1  Captain Barbossa, long believed to be dead, ha...  
2  A cryptic message from Bond’s past sends him o...  
3  Following the death of District Attorney Harve...  
4  John Carter is a war-weary, former military ca...  
Matriz TF-IDF creada con dimensión: (4803, 20978)
Películas recomendadas para 'The Godfather':

2731     The Godfather: Part II
1873                 Blood Ties
867     The Godfather: Part III
3727                 Easy Money
3623                       Made
3125                     Eulogy
3896                   Sinister
4506            The Maid's Room
3783                        Joe
224